<a href="https://colab.research.google.com/github/MasahiroAraki/SpeechRecognition/blob/master/Python/chap06a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 例題6.1

線形分離可能なデータに対するSVMの学習をscikit-learnでコーディングせよ．

データの準備と表示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.array([[1.0, 2.0], [1.0, 4.0], [2.0, 4.0], [2.0, 1.0], [5.0, 1.0], [4.0, 2.0]])
y = np.array([1, 1, 1, -1, -1, -1])
plt.grid()
plt.scatter(X[:, 0][y==1], X[:, 1][y==1], c='blue', label='class1')
plt.scatter(X[:, 0][y==-1], X[:, 1][y==-1], c='red', label='class2')
plt.legend(loc='upper right')

scikit-learnの線形SVCで学習

学習された識別面の係数を確認

In [ ]:
print(clf.coef_)
print(clf.intercept_)

## 例題6.2

線形分離可能なデータに対するSVMの学習をscikit-learnでコーディングせよ．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.array([[1.0, 4.0], [2.0, 3.0], [4.0, 3.0], [5.0, 4.0], [2.0, 1.0], [3.0, 2.0], [3.0, 3.0], [4.0, 1.0]])
y = np.array([1, 1, 1, 1, -1, -1, -1, -1])
plt.grid()
plt.scatter(X[:, 0][y==1], X[:, 1][y==1], c='blue', label='class1')
plt.scatter(X[:, 0][y==-1], X[:, 1][y==-1], c='red', label='class2')
plt.legend(loc='upper center')

scikit-learnのSVC（多項式カーネル）で学習

RBFカーネルで学習